In [1]:
import os
import pandas as pd
import numpy as np
from scipy.constants import value
from tqdm import tqdm

from src.utils.utils import get_logger
import logging

from IPython.display import clear_output

logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

In [2]:
# Leitura dos dados históricos

grouped_data = pd.read_parquet("../data/processed/average_unity_price_historic.parquet")

In [3]:
# Deleção dos dados nulos
logger.info("Filtrando e corrigindo dados")
grouped_data.dropna(inplace=True)
grouped_data = grouped_data[(grouped_data["importador_uf"] != "") \
                            & (grouped_data["importador_municipio"] != "")].copy()

2024-03-18 11:00:24,175 - __main__ - INFO - Filtrando e corrigindo dados


In [4]:

# Correção de estados e municipios
estados_br = ["AC", "AL", "AP", "AM", "BA", "CE", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE",
              "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO", "DF"]

grouped_data["old_municipio"] = grouped_data["importador_municipio"]

logger.info("\tajustando importador UF")
grouped_data["importador_uf_new"] = grouped_data.apply(lambda x: x["importador_uf"] \
    if x["importador_uf"] in estados_br else x["importador_municipio"], axis=1)

logger.info("\tajustando importador municipio")
grouped_data["importador_municipio_new"] = grouped_data.apply(lambda x: x["old_municipio"] \
    if x["old_municipio"] not in estados_br else x["importador_uf"], axis=1)

grouped_data.drop(columns=['importador_municipio', "importador_uf", "old_municipio"], inplace=True)
grouped_data.rename(columns={"importador_municipio_new": "importador_municipio",
                   "importador_uf_new": "importador_uf"}, inplace=True)

2024-03-18 11:00:32,249 - __main__ - INFO - 	ajustando importador UF
2024-03-18 11:01:17,940 - __main__ - INFO - 	ajustando importador municipio


In [5]:
# Criação das chaves agrupadas
grouped_data["key"] = grouped_data["ncm"].astype(str) + '-' + grouped_data["id_pais_origem"] + '-' + \
                      grouped_data['importador_municipio'] + '-' + grouped_data['urf']

In [6]:
# Filtro de chaves que tiveram preços unitários registrados em 2024
keys_2_check = grouped_data[grouped_data["ano_semestre"]==202302]["key"].unique()
grouped_2_check = grouped_data[grouped_data["key"].isin(keys_2_check)].copy()
grouped_2_check = grouped_2_check[["key", "semestre", "ano", "avg_valor_item"]].copy()
grouped_2_check["ano_semestre"] = grouped_2_check["ano"]*10 + grouped_2_check["semestre"]
grouped_2_check = grouped_2_check.groupby(["key", "ano_semestre"], as_index=False).mean("avg_valor_item")
# grouped_2_check["ano_semestre"] = grouped_2_check["ano_semestre"].astype(str)

In [7]:
# Pega do dado realizado interpolado
historic_data = pd.read_parquet("../data/processed/exp_g2/o2_trended_data_interpolated.parquet")
historic_data = historic_data[(historic_data["ano"]<2022) | (historic_data["semestre"]<2)]
historic_data["key"] = historic_data["ncm"].astype(str) + '-' + historic_data["id_pais_origem"] + '-' + \
                      historic_data['importador_municipio'] + '-' + historic_data['urf']

# Filtro do dados realizado interpolado somente para o que tem dado realizado em 2024
historic_data = historic_data[["key", "semestre", "ano","avg_valor_item"]].copy()
historic_data = historic_data[historic_data["key"].isin(keys_2_check)].copy()

# Junção do realizado interpolado + realizado em 2024
final_data = pd.concat([historic_data, grouped_2_check])
final_data["ano_semestre"] = final_data["ano"]*10 + final_data["semestre"]
final_data_gp = final_data.groupby(["key", "ano_semestre"], as_index=False).mean("avg_valor_item")
final_data_gp.sort_values(by=["key", "ano_semestre"], inplace=True)

# Pivot da tabela para facilitar a iteração dos grupos
final_data_pivot = final_data_gp.pivot(columns="ano_semestre", index="key", values="avg_valor_item")
final_data_pivot = final_data_pivot.dropna(subset=[20231, 20232])
# Aplicação da regra de queda ou aumento no valor unitário
final_data_pivot["trend_realizada"] = final_data_pivot.apply(lambda x: "aumento" if x[20231] < x[20232] else "queda",axis=1)

true_data = final_data_pivot[["trend_realizada"]].copy().reset_index()

In [9]:
# Checar as previsões
processed_data = pd.read_parquet("../data/processed/exp_g2/o2_trended_data_interpolated.parquet")
processed_data = processed_data[processed_data["ano"]>=2023]
processed_data["key"] = processed_data["ncm"].astype(str) + '-' + processed_data["id_pais_origem"] + '-' + \
                      processed_data['importador_municipio'] + '-' + processed_data['urf']

processed_data["ano_semestre"] = processed_data["ano"]*10 + processed_data["semestre"]
processed_data = processed_data.groupby(["key", "ano_semestre"], as_index=False).mean("avg_valor_item")
processed_data.sort_values(by=["key", "ano_semestre"], inplace=True)

In [10]:
processed_data_pivot = processed_data.pivot(columns="ano_semestre", index="key", values="avg_valor_item")
processed_data_pivot["trend_projetada"] = processed_data_pivot.apply(lambda x: "aumento" if x[20231] < x[20232]  else "queda",axis=1)
processed_data_trend = processed_data_pivot[["trend_projetada"]].copy().reset_index()
processed_data_trend.head()

ano_semestre,key,trend_projetada
0,1.0051E7-ALEMANHA-CAMBE-AEROPORTO INTERNACIONA...,queda
1,1.101001E7-ALEMANHA-GUARAPUAVA-PORTO DE PARANAGUA,queda
2,1.1052E7-ALEMANHA-SAO PAULO-PORTO DE SANTOS,aumento
3,1.10813E7-ALEMANHA-SAO PAULO-PORTO DE SANTOS,queda
4,1.10819E7-ALEMANHA-RIO DE JANEIRO-PORTO DO RIO...,aumento


In [11]:
# Merge data
trend_n_real = processed_data_trend.merge(true_data, on="key")
trend_n_real["score"] = trend_n_real.apply(lambda x: 1 if x["trend_projetada"] == x["trend_realizada"] else 0,axis=1) 
trend_n_real["score"].value_counts()/trend_n_real.shape[0]

Series([], Name: score, dtype: float64)

In [12]:
trend_n_real = processed_data_trend.merge(true_data, on="key")
trend_n_real[["trend_projetada", "trend_realizada"]].value_counts()

Series([], dtype: int64)

In [13]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
sns.heatmap(confusion_matrix(trend_n_real["trend_realizada"], trend_n_real["trend_projetada"]), annot=True, fmt='g')

ValueError: zero-size array to reduction operation fmin which has no identity

In [ ]:
trend_n_real["trend_realizada"].value_counts()

In [ ]:
trend_n_real["trend_projetada"].value_counts()